# Creating A GAN-Network

In [1]:
import tensorflow as tf
% pylab
% matplotlib inline 

C:\Users\Philipp\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


## Helperfunctions

In [2]:
def weight_variable(shape):
    initial = tf.glorot_uniform_initializer()
    return tf.Variable(initial(shape))

def bias_variable(shape):
    initial = tf.zeros.initializer()
    return tf.Variable(initial(shape))

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

def conv2d(x, W):
    return tf.nn.conv2d(x, filter=W, strides=[1, 1, 1, 1], padding='VALID')

def conv2d_transposed(x, filters): 
    return tf.layers.conv2d_transpose(inputs=x, filters=filters, kernel_size=[2, 2], strides=[2, 2], padding='VALID')

In [16]:
def createRedTestImage():
    image = np.zeros((300, 300, 3), dtype=np.float32)
    for x in range(300):
        for y in range(300):
            image[x, y] = np.array((
                           np.random.randint(230, 255) - np.random.random(), 
                           np.random.randint(0, 40) + np.random.random(),
                           np.random.randint(0, 40) + np.random.random()))
    return image



def createBlueTestImage():
    image = np.zeros((300, 300,3), dtype=np.float32)
    
    for x in range(300):
        for y in range(300):
            image[x, y] = np.array((
                            np.random.randint(0, 40) + np.random.random(), 
                            np.random.randint(0, 40) + np.random.random(),
                            np.random.randint(230, 255) - np.random.random()))
    return image
            
def getTrainSet(number=1000):
    training_set = [[0, 0] for _ in range(number)]
    for i in range(number):
        # Rot
        if np.random.rand() >= 0.5:
            training_set[i][0] = createRedTestImage()
            training_set[i][1] = np.array([1, 0])
        else:
            training_set[i][0] = createBlueTestImage()
            training_set[i][1] = np.array([0, 1])
    return training_set

def extractImagesAndLabes(training_set):
    pass

In [17]:
import time
startTime = time.time()
img = getTrainSet(100)
print(time.time() - startTime)

59.96351933479309


In [9]:
imgList = img
labels = np.asarray([pic[1] for pic in imgList])
images = np.asarray([pic[0] for pic in imgList])
len(labels), len(images)


[[[ 18  12 238]
  [ 32  27 246]
  [  0  12 240]
  ...
  [  1  16 239]
  [ 23  38 236]
  [ 23   8 245]]

 [[ 39   5 253]
  [ 15   5 231]
  [ 11  20 234]
  ...
  [ 32  34 248]
  [ 31  30 241]
  [  5   3 254]]

 [[ 26   1 246]
  [ 11  14 254]
  [  3  17 236]
  ...
  [ 25  15 249]
  [ 31  14 234]
  [  1  32 232]]

 ...

 [[ 10  16 240]
  [ 27   1 238]
  [ 30  27 252]
  ...
  [ 23   3 242]
  [ 36   3 234]
  [ 11  33 251]]

 [[  9  39 244]
  [ 18  33 237]
  [ 23  38 234]
  ...
  [ 38   4 237]
  [ 24  37 244]
  [ 38  36 252]]

 [[ 36  35 246]
  [ 25  12 245]
  [ 11  39 231]
  ...
  [  4   0 231]
  [ 15   2 243]
  [ 13   4 253]]] [0 1]


# Setup - 
## Discriminator

In [14]:
def discriminator(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    input_layer = tf.reshape(features["x"], [-1, 300, 300, 3])

    # Convolutional Layer #1
    # Computes 32 features using a 5x5 filter with ReLU activation.
    # Input Tensor Shape: [batch_size, 300, 300, 3]
    # Output Tensor Shape: [batch_size, 298, 298, 32]
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="VALID",
        activation=tf.nn.relu)
    
    # Convolutional Layer #2
    # Computes 32 features using a 5x5 filter with ReLU activation.
    # Input Tensor Shape: [batch_size, 298, 298, 32]
    # Output Tensor Shape: [batch_size, 296, 296, 32]
    conv2 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="VALID",
        activation=tf.nn.relu)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 296, 296, 32]
    # Output Tensor Shape: [batch_size, 148, 148, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Convolutional Layer #3
    # Computes 64 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 148, 148, 32]
    # Output Tensor Shape: [batch_size, 146, 146, 64]
    conv3 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="VALID",
        activation=tf.nn.relu)
    
    # Input Tensor Shape: [batch_size, 146, 146, 64]
    # Output Tensor Shape: [batch_size, 144, 144, 128]
    conv4 = tf.layers.conv2d(
        inputs=conv3,
        filters=128,
        kernel_size=[5, 5],
        padding="VALID",
        activation=tf.nn.relu)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 144, 144, 128]
    # Output Tensor Shape: [batch_size, 72, 72, 128]
    pool2 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)

    # Convolutional Layer #5
    # Computes 64 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 72, 72, 128]
    # Output Tensor Shape: [batch_size, 70, 70, 256]
    conv5 = tf.layers.conv2d(
        inputs=pool2,
        filters=256,
        kernel_size=[5, 5],
        padding="VALID",
        activation=tf.nn.relu)
    
    # Input Tensor Shape: [batch_size, 70, 70, 256]
    # Output Tensor Shape: [batch_size, 68, 68, 256]
    conv6 = tf.layers.conv2d(
        inputs=conv5,
        filters=256,
        kernel_size=[5, 5],
        padding="VALID",
        activation=tf.nn.relu)
    
    # Pooling Layer #3
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 68, 68, 256]
    # Output Tensor Shape: [batch_size, 34, 34, 256]
    pool3 = tf.layers.max_pooling2d(inputs=conv6, pool_size=[2, 2], strides=2)
    
    # Flatten tensor into a batch of vectors
    # Input Tensor Shape:  [batch_size, 34, 34, 256]
    # Output Tensor Shape: [batch_size, 34*34*256]
    pool3_flat = tf.reshape(pool3, [-1, 34 * 34 * 256])
    
    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 34*34*256]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool3_flat, units=1024, activation=tf.nn.relu)
    
    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
                inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
  
    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 2]
    logits = tf.layers.dense(inputs=dropout, units=2)
    
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [7]:
def main(unused_argv):
    # Load training and eval data
    # TODO: HIER DIE DATENSÄTZE LADEN
    train_data = images
    train_labels = labels
    eval_data = images
    eval_labels = labels
 
    # Create the Estimator
    cnn_classifier = tf.estimator.Estimator(model_fn=discriminator, model_dir='/tmp/mnist_convnet_model')

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)
    
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=5,
        num_epochs=None,
        shuffle=True)
    
    cnn_classifier.train(
        input_fn=train_input_fn,
        steps=20000,
        hooks=[logging_hook])
    
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [15]:
tf.app.run()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000235B0AB1518>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


TypeError: Value passed to parameter 'input' has DataType uint8 not in list of allowed values: float16, float32